## Movielens

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime

pd.set_option('display.float_format', lambda x : '{:,.2f}'.format(x))
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
if not ('sc' in locals() or 'sc' in globals()):
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
    
    conf = SparkConf()
    conf.setMaster('spark://spark-master:7077')
    conf.set('spark.executor.memory', '512m')
    conf.set('spark.app.name', 'basics')


    sc = SparkContext.getOrCreate(SparkContext(conf=conf))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

## Context
### Exercise 1
__We want to know the 25 most rated movie titles from this data. How many times a movie has been rated?__

#### Users
Users: This file name is kept as “u.user”, The columns in this file are:

```python
['user_id', 'age', 'sex', 'occupation', 'zip_code']
```

#### Ratings
Ratings: This file name is kept as “u.data”, The columns in this file are:

```python
['user_id', 'movie_id', 'rating', 'unix_timestamp']
```

#### Movies
Movies: This file name is kept as “u.item”, The columns in this file are:

```python
['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', and 18 more columns.....]
```

#### Load data
The firs step is to see how the data looks like

In [ ]:
userRDD = 

In [ ]:
ratingRDD = 

In [ ]:
movieRDD = 

#### Process

After looking the shape of the data we want to convert this raw data into a format easy to handle

In [ ]:
# Create a RDD from RatingRDD that only contains the two columns of interest i.e. movie_id,rating.
RDD_movid_rating = ratingRDD

In [ ]:
# Create a RDD from MovieRDD that only contains the two columns of interest i.e. movie_id,title.
RDD_movid_title = movieRDD

#### Merge

Merge these two pair RDDs based on movie_id. For this we will use the transformation [leftOuterJoin()](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.leftOuterJoin.html)

In [ ]:
rdd_movid_title_rating = RDD_movid_rating

Use the RDD in previous step to create (movie, 1) tuple pair RDD

In [ ]:
rdd_title_rating = rdd_movid_title_rating

Use the reduceByKey transformation to reduce on the basis of movie_title

In [ ]:
rdd_title_ratingcnt = rdd_title_rating

In [ ]:
for movie in rdd_title_ratingcnt

### Exercise 2

__Join all the sentences into a joined one__

In [ ]:
top = 

In [ ]:
for movies in top:
    print(movies)

### Exercise 3

__Now we want to find the most highly rated 25 movies using the same dataset. We actually want only those movies which have been rated at least 100 times.__

We already have the RDD rdd_movid_title_rating:

In [ ]:
rdd_movid_title_rating.take(2)

We create an RDD that contains sum of all the ratings for a particular movie

In [ ]:
rdd_title_ratingsum = 

Merge this data with the RDD rdd_title_ratingcnt we created in the last step


And use Map function to divide ratingsum by rating count.

In [ ]:
rdd_title_ratingmean_rating_count = 

We could use take ordered here only but we want to only get the movies which have count

of ratings more than or equal to 100 so lets filter the data RDD.

In [ ]:
rdd_title_rating_rating_count_gt_100 = 

In [ ]:
print("rdd_title_rating_rating_count_gt_100: ", rdd_title_rating_rating_count_gt_100.take(1))


In [ ]:
print ("25 highly rated movies:")
for i in rdd_title_rating_rating_count_gt_100.takeOrdered(25, lambda x:-x[1][0]):
    print(i)